In [25]:
import os
import pickle
import pydicom
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pydicom.pixel_data_handlers.util import apply_voi_lut

In [26]:
lst_points = [[11, [[-33.32080078125, 13.382699966431, -20.75119972229],
                [-16.425800323486, 13.382699966431, -20.75119972229],
                [-16.425800323486, 33.547698974609, -20.75119972229],
                [-33.32080078125, 33.547698974609, -20.75119972229]]],
              [13, [[-33.32080078125, 13.382699966431, -6.3511900901794],
                [-16.425800323486, 13.382699966431, -6.3511900901794],
                [-16.425800323486, 33.547698974609, -6.3511900901794],
                [-33.32080078125, 33.547698974609, -6.3511900901794]]],
              [17, [[-40.405799865723, -32.397399902344, 15.24880027771],
                [56.059299468994, -32.397399902344, 15.24880027771],
                [56.059299468994, 65.15779876709, 15.24880027771],
                [-40.405799865723, 65.15779876709, 15.24880027771]]]]


In [27]:
pixel_spacing = 0.4492

In [28]:
listImagePositionPatient = [[-107.387, -106.245, -85.5513],
                            [-107.794, -106.074, -79.5675],
                            [-108.2, -105.904, -73.5837],
                            [-108.607, -105.733, -67.6],
                            [-109.014, -105.562, -61.6162],
                            [-109.421, -105.391, -55.6324],
                            [-109.828, -105.22, -49.6487],
                            [-110.234, -105.049, -43.6649],
                            [-110.641, -104.879, -37.6811],
                            [-111.048, -104.708, -31.6974],
                            [-111.455, -104.537, -25.7136],
                            [-111.862, -104.366, -19.7298],
                            [-112.268, -104.195, -13.7461],
                            [-112.675, -104.025, -7.76232],
                            [-113.082, -103.854, -1.77855],
                            [-113.489, -103.683, 4.20521],
                            [-113.896, -103.512, 10.189],
                            [-114.302, -103.341, 16.1727],
                            [-114.709, -103.17, 22.1565],
                            [-115.116, -103, 28.1403],
                            [-115.523, -102.829, 34.124],
                            [-115.93, -102.658, 40.1078],
                            [-116.336, -102.487, 46.0916],
                            [-116.743, -102.316, 52.0753],
                            [-117.15, -102.146, 58.0591]]

In [29]:
def find_point_in_slice(point3D1, point3D2, z_axis):
    """
    Intersection point between plane and line in 3D-dimensions
    Input: 
    Line in 3d-dimensions 
    - point3D1: [x1,y1,z1]
    - point3D2: [x2,y2,z2]
    Plane in 3D-dimensions 
    - z_axis: z = z0
    Output:
    Intersection points
    """
    inter_x = (z_axis-point3D1[2])/(point3D2[2]-point3D1[2])*(point3D2[0]-point3D1[0]) + point3D1[0]
    inter_y = (z_axis-point3D1[2])/(point3D2[2]-point3D1[2])*(point3D2[1]-point3D1[1]) + point3D1[1]
    inter_point = [inter_x, inter_y, z_axis]
    return inter_point


In [30]:
def covert_coordinate(listPoints3D, pointRoot3D, pixelSpacing):
    """
    Function convert real coordiantes points 3D into point 2D
    Input:
    @param: listPoints3D: List of list 4 points real world coordinates [[x0,y0,z0],[x1,y1,z1],[x2,y2,z2],[x3,y3,z3]]
    @param: pointRoot3D: point 3D ImagePositionPatient [x,y,z]
    @param: pixelSpacing
    Output:
    @param: plot_point: bounding boxes (x,y,w,h) for convenience to plot with function Rectangle and to object detection model later
    """
    # delta = np.sqrt(2*((pixelSpacing/2)**2))
    # pointRoot3D[0] = pointRoot3D[0] + delta
    # pointRoot3D[1] = pointRoot3D[1] + delta

    pt2D = []
    for pt in listPoints3D:
        new_pt = [(pt[0] - pointRoot3D[0])/pixelSpacing,
                  (pt[1] - pointRoot3D[1])/pixelSpacing]
        pt2D.append(new_pt)

    x = [p[0] for p in pt2D]
    y = [p[1] for p in pt2D]
    bottomleftx = min(x)
    bottomlefty = min(y)
    height = max(y) - min(y)
    width = max(x) - min(x)
    bbox = [bottomleftx, bottomlefty, width, height]
    return bbox

In [31]:
def find_missing_slices(listPoints3D, listImagePositionPatient, pixel_spacing):
    """
    Find missing slices between 3 given slices
    Input: 
    - listPoints3D: List of index slices and 3D-points in these slices [[index1, [list4Points1]],[index2, [list4Points2]], [index3, [list4Points3]]]
    - listImagePositionPatient:
    - pixelSpacing: 
    Output:
    - listSlices: List of list target index slices and 3D-points in these slices [[index1_, [list4Points1_]],[index2_, [list4Points2_]], [index3_, [list4Points3_]],...]
    """
    # Slices Upper, Middle, Lower
    list_index_slices_up_mid_low = [item[0] for item in listPoints3D]
    listRootPointsUpMidLow = [listImagePositionPatient[idx] for idx in list_index_slices_up_mid_low]
    points3D = [item[1] for item in listPoints3D]
    upper4Points = points3D[0]
    middle4Points = points3D[1]
    lower4Points = points3D[2]

    slicesUpMidLow = [[list_index_slices_up_mid_low[i], 
                    covert_coordinate(points3D[i], listRootPointsUpMidLow[i], pixel_spacing)] for i in range(len(list_index_slices_up_mid_low))]

    index_upper_slice = list_index_slices_up_mid_low[0]
    index_middle_slice = list_index_slices_up_mid_low[1]
    index_lower_slice = list_index_slices_up_mid_low[2]

    # Loop each plane Upper -> Middle 
    list_missing_slices = [] 
    for index1 in range(index_upper_slice+1, index_middle_slice, 1):
        missing_slice_z_axis = listImagePositionPatient[index1][2]
        # Loop each point in set 4-points
        points_real_world_coordinates_in_slices = [find_point_in_slice(upper4Points[i], middle4Points[i], missing_slice_z_axis) for i in range(len(middle4Points))]
        points_in_slices = covert_coordinate(
                        listPoints3D=points_real_world_coordinates_in_slices,
                        pointRoot3D=listImagePositionPatient[index1],
                        pixelSpacing=pixel_spacing
                        )
        list_missing_slices.append([index1, points_in_slices])

    # Loop each plane Middle -> Lower 
    for index2 in range(index_middle_slice+1, index_lower_slice, 1):
        missing_slice_z_axis = listImagePositionPatient[index2][2]
        # Loop each point in set 4-points
        points_real_world_coordinates_in_slices = [find_point_in_slice(middle4Points[i], lower4Points[i], missing_slice_z_axis) for i in range(len(middle4Points))]
        points_in_slices = covert_coordinate(
                        listPoints3D=points_real_world_coordinates_in_slices,
                        pointRoot3D=listImagePositionPatient[index2],
                        pixelSpacing=pixel_spacing
                        )
        list_missing_slices.append([index2, points_in_slices])
    # Append Upper, Middle, Lower slices
    list_missing_slices.extend(slicesUpMidLow)
    return list_missing_slices

In [32]:
# Test functions
target_plot_planes = find_missing_slices(lst_points, listImagePositionPatient, pixel_spacing)
target_plot_planes

[[12,
  [175.75066611475958,
   261.74910945331925,
   37.611310012831694,
   44.89091497813445]],
 [14,
  [174.2237988755269,
   239.41499970452668,
   75.11064899760476,
   81.36292358634893]],
 [15,
  [170.76046560125624,
   210.8012859677697,
   124.1823100551716,
   129.09021939915525]],
 [16,
  [167.29711042075022,
   182.1874306827037,
   173.25421713694934,
   176.81775449610305]],
 [11,
  [174.84683708537398,
   262.12978621200136,
   37.61131001283172,
   44.89091497813445]],
 [13,
  [176.65672132402048,
   261.3706588745125,
   37.611310012831694,
   44.89091497813445]],
 [17,
  [164.50623360257572,
   157.9332148211398,
   214.74866281103516,
   217.17542001209705]]]

In [33]:
list_index_slices_up_mid_low = [item[0] for item in lst_points]

listRootPoints = [listImagePositionPatient[idx] for idx in list_index_slices_up_mid_low]
RootPointsUpper = listRootPoints[0]
RootPointsMiddle = listRootPoints[1]
RootPointsLower = listRootPoints[2]

points = [item[1] for item in lst_points]
upper4Points = points[0]
middle4Points = points[1]
lower4Points = points[2]

slicesUpMidLow = [[list_index_slices_up_mid_low[i], 
                    covert_coordinate(points[i], listRootPoints[i], pixel_spacing)] for i in range(len(lst_points))]
slicesUpMidLow

[[11,
  [174.84683708537398,
   262.12978621200136,
   37.61131001283172,
   44.89091497813445]],
 [13,
  [176.65672132402048,
   261.3706588745125,
   37.611310012831694,
   44.89091497813445]],
 [17,
  [164.50623360257572,
   157.9332148211398,
   214.74866281103516,
   217.17542001209705]]]

In [34]:
index_upper_slice = list_index_slices_up_mid_low[0]
index_middle_slice = list_index_slices_up_mid_low[1]
index_lower_slice = list_index_slices_up_mid_low[2]

# Loop each plane
list_missing_slices = [] 
for index1 in range(index_upper_slice+1, index_middle_slice, 1):
    # print(index1)
    missing_slice_z_axis = listImagePositionPatient[index1][2]
    # print(missing_slice_z_axis)
    # Loop each point in set 4-points
    points_real_world_coordinates_in_slices = [find_point_in_slice(upper4Points[i], middle4Points[i], missing_slice_z_axis) for i in range(len(middle4Points))]
    points_in_slices = covert_coordinate(
                    listPoints3D=points_real_world_coordinates_in_slices,
                    pointRoot3D=listImagePositionPatient[index1],
                    pixelSpacing=pixel_spacing
                    )
    list_missing_slices.append([index1, points_in_slices])

print(list_missing_slices)


[[12, [175.75066611475958, 261.74910945331925, 37.611310012831694, 44.89091497813445]]]


In [35]:
for index2 in range(index_middle_slice+1, index_lower_slice, 1):
    # print(index2)
    missing_slice_z_axis = listImagePositionPatient[index2][2]
    # print(missing_slice_z_axis)
    # Loop each point in set 4-points
    points_real_world_coordinates_in_slices = [find_point_in_slice(middle4Points[i], lower4Points[i], missing_slice_z_axis) for i in range(len(middle4Points))]
    points_in_slices = covert_coordinate(
                    listPoints3D=points_real_world_coordinates_in_slices,
                    pointRoot3D=listImagePositionPatient[index2],
                    pixelSpacing=pixel_spacing
                    )
    list_missing_slices.append([index2, points_in_slices])

print(list_missing_slices)


[[12, [175.75066611475958, 261.74910945331925, 37.611310012831694, 44.89091497813445]], [14, [174.2237988755269, 239.41499970452668, 75.11064899760476, 81.36292358634893]], [15, [170.76046560125624, 210.8012859677697, 124.1823100551716, 129.09021939915525]], [16, [167.29711042075022, 182.1874306827037, 173.25421713694934, 176.81775449610305]]]


In [36]:
# Append 3 slices Upper, Middle, Lower 
list_missing_slices.extend(slicesUpMidLow)
list_missing_slices

[[12,
  [175.75066611475958,
   261.74910945331925,
   37.611310012831694,
   44.89091497813445]],
 [14,
  [174.2237988755269,
   239.41499970452668,
   75.11064899760476,
   81.36292358634893]],
 [15,
  [170.76046560125624,
   210.8012859677697,
   124.1823100551716,
   129.09021939915525]],
 [16,
  [167.29711042075022,
   182.1874306827037,
   173.25421713694934,
   176.81775449610305]],
 [11,
  [174.84683708537398,
   262.12978621200136,
   37.61131001283172,
   44.89091497813445]],
 [13,
  [176.65672132402048,
   261.3706588745125,
   37.611310012831694,
   44.89091497813445]],
 [17,
  [164.50623360257572,
   157.9332148211398,
   214.74866281103516,
   217.17542001209705]]]